Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [12]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    print("Shape of conv2", shape)
    print("Shape of layer3_weights", layer3_weights.get_shape().as_list())
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Shape of conv2 [16, 7, 7, 16]
Shape of layer3_weights [784, 64]
Shape of conv2 [10000, 7, 7, 16]
Shape of layer3_weights [784, 64]
Shape of conv2 [10000, 7, 7, 16]
Shape of layer3_weights [784, 64]


In [13]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.222507
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.677825
Minibatch accuracy: 37.5%
Validation accuracy: 47.9%
Minibatch loss at step 100: 0.547104
Minibatch accuracy: 87.5%
Validation accuracy: 53.9%
Minibatch loss at step 150: 1.163646
Minibatch accuracy: 68.8%
Validation accuracy: 55.5%
Minibatch loss at step 200: 0.965720
Minibatch accuracy: 62.5%
Validation accuracy: 58.0%
Minibatch loss at step 250: 0.980884
Minibatch accuracy: 68.8%
Validation accuracy: 60.1%
Minibatch loss at step 300: 0.293476
Minibatch accuracy: 87.5%
Validation accuracy: 60.9%
Minibatch loss at step 350: 0.268455
Minibatch accuracy: 93.8%
Validation accuracy: 61.6%
Minibatch loss at step 400: 0.452523
Minibatch accuracy: 87.5%
Validation accuracy: 60.5%
Minibatch loss at step 450: 0.671579
Minibatch accuracy: 75.0%
Validation accuracy: 62.9%
Minibatch loss at step 500: 0.822679
Minibatch accuracy: 75.0%
Validation accuracy: 63.6%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [15]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool2 = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = pool2.get_shape().as_list()
    reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.743438
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.379901
Minibatch accuracy: 18.8%
Validation accuracy: 17.3%
Minibatch loss at step 100: 1.537967
Minibatch accuracy: 43.8%
Validation accuracy: 31.8%
Minibatch loss at step 150: 1.180764
Minibatch accuracy: 56.2%
Validation accuracy: 46.6%
Minibatch loss at step 200: 0.807281
Minibatch accuracy: 75.0%
Validation accuracy: 58.5%
Minibatch loss at step 250: 0.848645
Minibatch accuracy: 68.8%
Validation accuracy: 58.2%
Minibatch loss at step 300: 0.376494
Minibatch accuracy: 93.8%
Validation accuracy: 63.7%
Minibatch loss at step 350: 0.254350
Minibatch accuracy: 93.8%
Validation accuracy: 63.0%
Minibatch loss at step 400: 0.398740
Minibatch accuracy: 87.5%
Validation accuracy: 63.7%
Minibatch loss at step 450: 0.703671
Minibatch accuracy: 75.0%
Validation accuracy: 65.7%
Minibatch loss at step 500: 0.883284
Minibatch accuracy: 81.2%
Validation accuracy: 66.8%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [66]:
batch_size = 128
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(None, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  c1_weights = tf.Variable(tf.truncated_normal(
      [5, 5, num_channels, 6], stddev=0.1))
  c1_biases = tf.Variable(tf.constant(1.0, shape=[6]))
  c3_weights = tf.Variable(tf.truncated_normal(
      [5, 5, 6, 16], stddev=0.1))
  c3_biases = tf.Variable(tf.constant(1.0, shape=[16]))
  c5_weights = tf.Variable(tf.truncated_normal(
      [4, 4, 16, 120], stddev=0.1))
  c5_biases = tf.Variable(tf.constant(1.0, shape=[120]))
  f6_weights = tf.Variable(tf.truncated_normal(
      [120, 84], stddev=0.1))
  f6_biases = tf.Variable(tf.constant(1.0, shape=[84]))
  output_weights = tf.Variable(tf.truncated_normal(
      [84, 10], stddev=0.1))
  output_biases = tf.Variable(tf.constant(1.0, shape=[10]))

  keep_prob = tf.placeholder(tf.float32)

  
  # Model.
  def model(data):
    c1 = tf.nn.dropout(tf.nn.relu(tf.nn.conv2d(data, c1_weights, [1, 1, 1, 1], padding='VALID') + c1_biases), keep_prob)
    s2 = tf.nn.avg_pool(c1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    c3 = tf.nn.dropout(tf.nn.relu(tf.nn.conv2d(s2, c3_weights, [1, 1, 1, 1], padding='VALID') + c3_biases), keep_prob)
    s4 = tf.nn.avg_pool(c3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    c5 = tf.nn.dropout(tf.nn.relu(tf.nn.conv2d(s4, c5_weights, [1, 1, 1, 1], padding='VALID') + c5_biases), keep_prob)
    shape = c5.get_shape().as_list()
    flat_c5 = tf.reshape(c5, [-1, shape[1] * shape[2] * shape[3]])
    f6 = tf.nn.dropout(tf.nn.relu(tf.matmul(flat_c5, f6_weights) + f6_biases), keep_prob)
    output = tf.matmul(f6, output_weights) + output_biases
    return output
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.25, global_step, 1000, 0.65, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 1.0}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      #print('batch label:',  batch_labels)
      #print('predictions:',  predictions)
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        train_prediction.eval({tf_train_dataset : valid_dataset, keep_prob: 1.0}), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(train_prediction.eval({tf_train_dataset : test_dataset, keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 4.173853
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.505750
Minibatch accuracy: 85.2%
Validation accuracy: 64.9%
Minibatch loss at step 1000: 0.465589
Minibatch accuracy: 85.9%
Validation accuracy: 71.1%
Minibatch loss at step 1500: 0.352268
Minibatch accuracy: 87.5%
Validation accuracy: 73.2%
Minibatch loss at step 2000: 0.458727
Minibatch accuracy: 85.9%
Validation accuracy: 74.0%
Minibatch loss at step 2500: 0.327149
Minibatch accuracy: 87.5%
Validation accuracy: 74.6%
Minibatch loss at step 3000: 0.276013
Minibatch accuracy: 91.4%
Validation accuracy: 75.7%
Minibatch loss at step 3500: 0.334352
Minibatch accuracy: 89.8%
Validation accuracy: 75.5%
Minibatch loss at step 4000: 0.320718
Minibatch accuracy: 90.6%
Validation accuracy: 76.0%
Minibatch loss at step 4500: 0.253048
Minibatch accuracy: 93.0%
Validation accuracy: 76.5%
Minibatch loss at step 5000: 0.439652
Minibatch accuracy: 84.4%
Validation accuracy

---
Trying dropout below
---------

In [78]:
batch_size = 128
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(None, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  c1_weights = tf.Variable(tf.truncated_normal(
      [5, 5, num_channels, 6], stddev=0.1))
  c1_biases = tf.Variable(tf.constant(1.0, shape=[6]))
  c3_weights = tf.Variable(tf.truncated_normal(
      [5, 5, 6, 16], stddev=0.1))
  c3_biases = tf.Variable(tf.constant(1.0, shape=[16]))
  c5_weights = tf.Variable(tf.truncated_normal(
      [4, 4, 16, 120], stddev=0.1))
  c5_biases = tf.Variable(tf.constant(1.0, shape=[120]))
  f6_weights = tf.Variable(tf.truncated_normal(
      [120, 84], stddev=0.1))
  f6_biases = tf.Variable(tf.constant(1.0, shape=[84]))
  output_weights = tf.Variable(tf.truncated_normal(
      [84, 10], stddev=0.1))
  output_biases = tf.Variable(tf.constant(1.0, shape=[10]))

  keep_prob = tf.placeholder(tf.float32)
  keep_prob_fc = tf.placeholder(tf.float32)


  
  # Model.
  def model(data):
    c1 = tf.nn.dropout(tf.nn.relu(tf.nn.conv2d(data, c1_weights, [1, 1, 1, 1], padding='VALID') + c1_biases), keep_prob)
    s2 = tf.nn.avg_pool(c1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    c3 = tf.nn.dropout(tf.nn.relu(tf.nn.conv2d(s2, c3_weights, [1, 1, 1, 1], padding='VALID') + c3_biases), keep_prob)
    s4 = tf.nn.avg_pool(c3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    c5 = tf.nn.dropout(tf.nn.relu(tf.nn.conv2d(s4, c5_weights, [1, 1, 1, 1], padding='VALID') + c5_biases), keep_prob)
    shape = c5.get_shape().as_list()
    flat_c5 = tf.reshape(c5, [-1, shape[1] * shape[2] * shape[3]])
    f6 = tf.nn.dropout(tf.nn.relu(tf.matmul(flat_c5, f6_weights) + f6_biases), keep_prob_fc)
    output = tf.matmul(f6, output_weights) + output_biases
    return output
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.25, global_step, 1000, 0.65, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 1.0, keep_prob_fc: 1.0}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      #print('batch label:',  batch_labels)
      #print('predictions:',  predictions)
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        train_prediction.eval({tf_train_dataset : valid_dataset, keep_prob: 1.0, keep_prob_fc: 1.0}), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(train_prediction.eval({tf_train_dataset : test_dataset, keep_prob: 1.0, keep_prob_fc: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 4.119660
Minibatch accuracy: 7.8%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.463929
Minibatch accuracy: 85.9%
Validation accuracy: 68.5%
Minibatch loss at step 1000: 0.435480
Minibatch accuracy: 85.9%
Validation accuracy: 73.1%
Minibatch loss at step 1500: 0.367854
Minibatch accuracy: 89.8%
Validation accuracy: 74.5%
Minibatch loss at step 2000: 0.378528
Minibatch accuracy: 89.1%
Validation accuracy: 75.0%
Minibatch loss at step 2500: 0.339636
Minibatch accuracy: 88.3%
Validation accuracy: 76.2%
Minibatch loss at step 3000: 0.268065
Minibatch accuracy: 89.8%
Validation accuracy: 76.8%
Minibatch loss at step 3500: 0.311474
Minibatch accuracy: 89.1%
Validation accuracy: 76.5%
Minibatch loss at step 4000: 0.334061
Minibatch accuracy: 91.4%
Validation accuracy: 77.1%
Minibatch loss at step 4500: 0.203068
Minibatch accuracy: 95.3%
Validation accuracy: 77.8%
Minibatch loss at step 5000: 0.443453
Minibatch accuracy: 85.2%
Validation accuracy

Tried dropout on different layers. Not working.